In [2]:
# reading in the EMAILS document as a textfile

with open('\Emails.txt', 'r') as textfile:
    # splitting at instance of 'Date Received:'
    text = textfile.read().split('Date Received: ')
    print(text[:3])
    # creating list for all fields
    Date_Received = []
    Response_Date = []
    Subject = []
    Body = []
    for item in text:
        # splitting the remaining text in each occurrence (all four fields appear in each instance of email)
        # appending each field with the corresponding text in each email
        tail = item.strip().split('Response Date:', 1)
#         if len(tail) == 1:
#             print(item)
        Date_Received.append('' if len(tail) == 1 else tail[0].strip())
        tail = tail[-1].split('Subject:', 1)
        if Date_Received[-1] == '':
            Date_Received[-1] = tail[0].strip()
            Response_Date.append('')
        else:
            Response_Date.append('' if len(tail) == 1 else tail[0].strip())
        tail = tail[-1].split('\n', 1)
        Subject.append('' if len(tail) == 1 else tail[0].strip())
        Body.append(tail[-1].strip())
        
Date_Received

FileNotFoundError: [Errno 2] No such file or directory: '\\Emails.txt'

In [ ]:
# Create dataframe from multiple lists
# Date_Received, Response_Date, Subject, Body

# 1. zip lists to create a list of tuples
zippedList =  list(zip(Date_Received, Response_Date, Subject, Body))

# 2. create a dataframe with the zipped lists
dfObj = pd.DataFrame(zippedList, columns = ['Date_Received', 'Response_Date', 'Subject', 'Body'])  #add index?

dfObj

In [3]:
#importing data and required libraries
import pandas as pd  
import numpy as np
import io

import gensim
import gensim.corpora as corpora
import re, nltk, spacy


# rename df
emails = dfObj

# remove first row of df
emails = emails.iloc[1:]


NameError: name 'dfObj' is not defined

In [ ]:
emails

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Convert to list
data = emails.Body.values.tolist()
data

In [ ]:
#Tokenize and Clean-up using gensim’s simple_preprocess()

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

In [ ]:
# lemmatization

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

In [ ]:
#creating vocabulary of all words in data
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text

count_vect = CountVectorizer(min_df=2, stop_words= text.ENGLISH_STOP_WORDS.union(["redact", "nan", 
                                                    "www", "https", "http", "com", "request", "org", "http", "ish",
                                                                                  "rss","lda"]))

# document term matrix
data_vectorized = count_vect.fit_transform(data_lemmatized)

In [ ]:
data_vectorized

In [ ]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

In [4]:
from sklearn.decomposition import LatentDirichletAllocation

# sentiment analysis revealed only 4 applicable topics. so running with 4
LDA = LatentDirichletAllocation(n_topics=4, random_state=42, learning_decay=0.5)  
LDA.fit(data_vectorized) 

NameError: name 'data_vectorized' is not defined

In [ ]:
# Define Search Param
# Don't have to do this every time, ideally doing this once, then refer to above block
from sklearn.decomposition import LatentDirichletAllocation

search_params = {'n_topics': [4, 5, 6, 7, 10], 'learning_decay': [.5, .7, .9]}

#, 'learning_decay': [.7]

# Init the Model
LDA2 = LatentDirichletAllocation(random_state=42)  

# Init Grid Search Class
model = GridSearchCV(LDA2, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)

In [5]:
#see best topic model
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

NameError: name 'model' is not defined

In [6]:
#printing top 10 words with highest probabilities for all topics
for i,topic in enumerate(LDA.components_):  
    print('Top 10 words for topic {}:'.format(i))
    print([count_vect.get_feature_names()[i] for i in topic.argsort()[::-1][:10]])
    print('\n')

AttributeError: 'LatentDirichletAllocation' object has no attribute 'components_'

In [7]:
#10 words with highest probability for first topic
first_topic = LDA.components_[0]
second_topic = LDA.components_[1]
third_topic = LDA.components_[2]
fourth_topic = LDA.components_[3]

AttributeError: 'LatentDirichletAllocation' object has no attribute 'components_'

In [8]:
#generate top 10 words for topic by probability
top_topic_words = first_topic.argsort()[-10:]
top_topic_words

NameError: name 'first_topic' is not defined

In [9]:
#retrieve value of words from count_vect object
for i in top_topic_words:  
    print(count_vect.get_feature_names()[i])

NameError: name 'top_topic_words' is not defined

In [30]:
#retrieve value of words from count_vect object
for i in top_topic_words:  
    print(count_vect.get_feature_names()[i])

study
year
antibiotic
test
borrelia
treatment
patient
infection
lyme
disease


In [31]:
emails.shape

(945, 4)

In [32]:
doc_topic_dist = LDA.transform(data_vectorized)
doc_topic_dist

array([[9.86067003e-03, 9.69469744e-01, 9.74635879e-03, 1.09232267e-02],
       [6.95513824e-02, 3.52238087e-01, 1.02986736e-01, 4.75223795e-01],
       [2.10829261e-02, 9.34409233e-01, 2.11424681e-02, 2.33653729e-02],
       ...,
       [7.22888811e-03, 4.72108234e-01, 7.44997573e-03, 5.13212902e-01],
       [5.42300397e-01, 4.56248810e-01, 7.10795726e-04, 7.39996587e-04],
       [9.89723299e-01, 7.56572419e-04, 7.28999376e-04, 8.79112911e-03]])

In [70]:
# attributes of best_lda_model from grid search
best_lda_model

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.5, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_jobs=1, n_topics=5, perp_tol=0.1,
             random_state=42, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [33]:
# Create Document - Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_topics)]

# index names
docnames = ["Doc" + str(i) for i in range(len(data))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,dominant_topic
Doc0,0.01,0.97,0.01,0.01,1
Doc1,0.07,0.35,0.1,0.48,3
Doc2,0.02,0.93,0.02,0.02,1
Doc3,0.39,0.41,0,0.2,1
Doc4,0.01,0.32,0.01,0.66,3
Doc5,0,0.28,0,0.71,3
Doc6,0,0.3,0,0.69,3
Doc7,0.01,0.33,0.01,0.66,3
Doc8,0.01,0.01,0.01,0.98,3
Doc9,0,0.25,0,0.74,3


In [36]:
#Sentiments for each topics: analysis to figure out which words drive the sentiment up (most associated with it)

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()

In [37]:
#polarity scores

def print_sentiment_scores(sentence):
    snt = analyser.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(snt)))
    return snt

In [35]:
# can normally skip
# one method of producing sentiment keys

sentence=['sample_text']
def print_sentiment_scores2(sentence):
    ss = analyser.polarity_scores(sentence)
#     print(ss)
    return ss

dict = print_sentiment_scores2('sample_text')
maxVal = 0
keyVal = ''
for key in dict:
    val = dict[key]
    if(val > maxVal):
        keyVal = key
        maxVal = val
print(maxVal)
print(keyVal)

0.91
neu


In [10]:
# FOR RUNNING BASIC POS OR NEG ASSIGNMENT (WITHOUT PRINTED SENTIMENT KEYS AND VALUES)

for index, row in emails.iterrows():
    dict = print_sentiment_scores(row['Body'])
    maxVal = 0
    keyVal = ''
    for key in dict:
        val = dict[key]
        if(val > maxVal):
            keyVal = key
            maxVal = val
            # assign new key and values of max value for each row
            emails['SentimentKey'] = keyVal
            emails['SentimentVal'] = maxVal

NameError: name 'emails' is not defined

In [1]:
# FOR RUNNING BASIC POS OR NEG ASSIGNMENT (WITH PRINTED SENTIMENT KEYS AND VALUES)

# for each row in emails 'Body' column, create a dictionary to contain respective sentiment score
for index, row in emails.iterrows():
    # create dictionary
    dict = print_sentiment_scores(row['Body'])
    maxVal = 0
    keyVal = ''
    # taking only positive or negative values from the sentiment score for each body
    valid = set(['pos', 'neg'])
    print("")
    print(dict)
    print("")
    print(valid)
    print("")
    maxVal = 0
    maxKey = ''
    # for each sentiment score per row with Body, find the maximum score
    for key in valid:
        val = dict[key]
        if (val > maxVal):
            maxVal = val
            maxKey = key
    print(maxKey)
    print(maxVal)

    emails.loc[index, 'SentimentKey'] = maxKey
    emails.loc[index, 'SentimentVal'] = maxVal

NameError: name 'emails' is not defined

In [7]:
emails

NameError: name 'emails' is not defined

In [8]:
# add a column to the original data frame that will store the topic for the text
topic_values = LDA.transform(data_vectorized)  
topic_values.shape

emails.shape

emails['Topic'] = topic_values.argmax(axis=1)
emails

NameError: name 'LDA' is not defined

In [107]:
# proportion of positive and negative comments in selected data st

topics = {}
for email in emails:
    if topics[email['Topic']]:
        topic = topics[email['Topic']]
        if email['SentimentKey'] == 'pos':
            topics[topic] = topics[topic] + 1 / 2
# Percentage calculation for SentimentKey
# initial calculation.....67% negative

(emails['SentimentKey'].value_counts()/emails['SentimentKey'].count())*100

neg    66.981132
pos    32.075472
        0.943396
Name: SentimentKey, dtype: float64

In [47]:
# Percentage calculation for SentimentKey by each topic
# how do I do this?

for i in np.unique(emails['Topic']):
    #counts = emails['Topic'].value_counts().values.tolist()
    pct = (emails['Topic'].value_counts()/emails['Topic'].count())*100

    

pct

3    54.285714
1    30.582011
0    11.534392
2     3.597884
Name: Topic, dtype: float64

In [89]:
# proportion of emails with positive sentiment across each topic

for name, group in emails.groupby('Topic')['SentimentKey']:
    print("name = " + str(name))
    countPos = 0
    for sign in group:
        if sign == 'pos':
            countPos+=1
    print("group " + str(name) + " = " + str(countPos / group.count() * 100) + "% pos")
    print(group.count())

name = 0
group 0 = 0.0% pos
109
name = 1
group 1 = 0.0% pos
289
name = 2
group 2 = 0.0% pos
34
name = 3
group 3 = 0.0% pos
513


In [149]:
emails.groupby('Topic')['SentimentKey']

pct = (emails['Topic'].value_counts()/emails['Topic'].count())*100

(emails['SentimentKey'].value_counts()/emails['SentimentKey'].count())*100

In [6]:
# TINKERING WITH THE FIRST SUCCESSFUL ALTERATION TO THE SENTIMENT CLASSIFICATION
# Using a threshold of >0.5 or <-0.5 for 'compound' value
# Assignment of positive for compound score >0.5, negativer for <-0.5 and neutral for -0.5 < x < 0.5

emails2 = emails

for index, row in emails2.iterrows():
    dict = print_sentiment_scores(row['Body'])
    valid = set(['compound'])
    print("")
    print(dict)
    print("")
    print(valid)
    print("")
    # for each sentiment score per row with Body, and given criteria, assign the appropriate sentiment
    for key in valid:
        val = dict[key]
        if (val > 0.5):
            val = val
            key = 1
        elif (val < -0.5):
            val = val
            key = -1  
        else:
            val = val
            key = 0
    print(key)
    print(val)
    
    #work on using the other dataset 'emails'
    emails2.loc[index, 'SentimentKey'] = key
    emails2.loc[index, 'SentimentVal'] = val

NameError: name 'emails' is not defined

In [103]:
# assigning distribution of sentiments across each topic

print(pd.DataFrame({'Percentage': emails2.groupby(('Topic', 'SentimentKey')).size() / len(emails2)}))

                    Percentage
Topic SentimentKey            
0     -1              0.065608
       0              0.022222
       1              0.027513
1     -1              0.056085
       0              0.078307
       1              0.171429
2     -1              0.004233
       0              0.013757
       1              0.017989
3     -1              0.159788
       0              0.117460
       1              0.265608


/Users/amcurtis91/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:7: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  import sys


In [9]:
emails2

NameError: name 'emails2' is not defined

In [49]:
# Percentage calculation for SentimentKey across entire data set
# for function applied to each topic, look to previous assignment script (two blocks up)

for i in np.unique(emails['SentimentKey']):
    pct = (emails['SentimentKey'].value_counts()/emails['SentimentKey'].count())*100
    
pct

 1    48.253968
-1    28.571429
 0    23.174603
Name: SentimentKey, dtype: float64

In [4]:
# emails2 has different sentimental analysis schema - positive, negative and neutral

emails2

NameError: name 'emails2' is not defined

In [5]:
emails

NameError: name 'emails' is not defined

In [3]:
# (emails) subset data into UP TO November 15, 2018 AND November 16, 2018 and beyond

e_first = emails.iloc[0:825]
e_first

e_second = emails.iloc[825:]
e_second

NameError: name 'emails' is not defined

In [66]:
# sentiment distribution for first subset of emails (up to November 15, 2018)

for i in np.unique(e_first['SentimentKey']):
    pct = (e_first['SentimentKey'].value_counts()/e_first['SentimentKey'].count())*100
    
pct

pos    62.787879
neg    34.545455
        2.666667
Name: SentimentKey, dtype: float64

In [67]:
# sentiment distribution for second subset of emails (after November 15, 2018)

for i in np.unique(e_second['SentimentKey']):
    pct = (e_second['SentimentKey'].value_counts()/e_second['SentimentKey'].count())*100
    
pct

neg    48.333333
pos    45.833333
        5.833333
Name: SentimentKey, dtype: float64

In [2]:
# (emails2) subset data into UP TO November 15, 2018 AND November 16, 2018 and beyond

e2_first = emails2.iloc[0:825]
e2_first

e2_second = emails2.iloc[825:]
e2_second

NameError: name 'emails2' is not defined

In [93]:
# emails2 pt. 1 sentiment distribution

for i in np.unique(e2_first['SentimentKey']):
    pct = (e2_first['SentimentKey'].value_counts()/e2_first['SentimentKey'].count())*100
    
pct

 1    50.181818
-1    26.666667
 0    23.151515
Name: SentimentKey, dtype: float64

In [94]:
# emails2 pt. 2 sentiment distribution

for i in np.unique(e2_second['SentimentKey']):
    pct = (e2_second['SentimentKey'].value_counts()/e2_second['SentimentKey'].count())*100
    
pct

-1    41.666667
 1    35.000000
 0    23.333333
Name: SentimentKey, dtype: float64

In [119]:
pct_tot = (emails2['SentimentKey'].value_counts()/emails2['SentimentKey'].count())*100
pct_tot

 1    48.253968
-1    28.571429
 0    23.174603
Name: SentimentKey, dtype: float64

In [150]:
print(pd.DataFrame({'Percentage': e2_first.groupby(('Topic', 'SentimentKey')).size() / len(e2_first)}))

                    Percentage
Topic SentimentKey            
0     -1              0.036364
       0              0.015758
       1              0.024242
1     -1              0.058182
       0              0.082424
       1              0.179394
2     -1              0.004848
       0              0.014545
       1              0.020606
3     -1              0.167273
       0              0.118788
       1              0.277576


/Users/amcurtis91/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  """Entry point for launching an IPython kernel.
